# Einsum Puzzles
This Colab will teach you how to use numpy.einsum through examples.

In [1]:
import numpy as np

In [2]:
vector_length = 5
u = np.arange(vector_length)
print(f"{u=}")
print(f"{u.shape=}")

r = np.einsum("i->i", u)
print(f"{r=}")
print(f"{r.shape=}")

u=array([0, 1, 2, 3, 4])
u.shape=(5,)
r=array([0, 1, 2, 3, 4])
r.shape=(5,)
